# Implement a Q-learning algorithm


## Step 0: Import the dependencies
In this notebook I try to replicate the tutorial from [Thomas Simonini](https://medium.freecodecamp.org/diving-deeper-into-reinforcement-learning-with-q-learning-c18d0db58efe)

First, we need to import the libraries that we will need to create our agent. We use 3 libraries:
- Numpy for Qtable
- OpenAI Gym for our Taxi Environment
- Random to generate random numbers

__Description of the problem:__
There are four designated locations in the grid world indicated by R(ed), B(lue), G(reen), and Y(ellow). When the episode starts, the taxi starts off at a random square and the passenger is at a random location. The taxi drive to the passenger's location, pick up the passenger, drive to the passenger's destination (another one of the four specified locations), and then drop off the passenger. Once the passenger is dropped off, the episode ends. [more here](https://github.com/openai/gym/blob/master/gym/envs/toy_text/taxi.py)

__Observations:__
There are 500 discrete states since there are 25 taxi positions, 5 possible locations of the passenger (including the case when the passenger is the taxi), and 4 destination locations. 

__Actions:__ 
There are 6 discrete deterministic actions:
 - 0: move south
 - 1: move north
 - 2: move east 
 - 3: move west 
 - 4: pickup passenger
 - 5: dropoff passenger


__Rewards:__
There is a reward of -1 for each action and an additional reward of +20 for delievering the passenger. There is a reward of -10 for executing actions "pickup" and "dropoff" illegally.
    
__Rendering:__
 - blue: passenger
 - magenta: destination
 - yellow: empty taxi
 - green: full taxi
 - other letters: locations

In [6]:
import numpy as np
import gym
import random

## Step 1: Create the environment
- Here we'll create the Taxi environment.
- OpenAI Gym is a library composed of many environments that we can use to train our agents.

In [37]:
env = gym.make("Taxi-v2")
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [48]:
env.reset()
new_state, reward, done, info = env.step(env.action_space.sample())
def printInfo():
    print("**********************")
    print("new_state",new_state)
    print("reward",reward)
    print("done",done)
    print("info",info)

printInfo()
new_state, reward, done, info = env.step(env.action_space.sample())
printInfo()

new_state, reward, done, info = env.step(env.action_space.sample())
printInfo()

new_state, reward, done, info = env.step(env.action_space.sample())
printInfo()

**********************
new_state 223
reward -1
done False
info {'prob': 1.0}
**********************
new_state 223
reward -10
done False
info {'prob': 1.0}
**********************
new_state 243
reward -1
done False
info {'prob': 1.0}
**********************
new_state 243
reward -10
done False
info {'prob': 1.0}


## Step 2: Create the Q-Table and initialize it
- Now, we'll create our Q-table, to know how much rows (states) and columns (actions) we need, also we nee to calculate the action_size and state_size
- OpenAI gym provides us a way to do that: env.action_space.n and env.observation_space.n

In [49]:
action_size = env.action_space.n
print("Action size", action_size)

state_size = env.observation_space.n
print("State size", state_size)

Action size 6
State size 500


In [50]:
qtable = np.zeros((state_size, action_size))
print(qtable)
print("shape:",qtable.shape)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
shape: (500, 6)


## Step 3: Create the hyperparameters

Here, we'll specify the hyperparameters.

In [51]:
total_episodes = 50000              # Total episodes
total_test_episodes = 100           # Total test episodes
max_steps = 99                      # Max steps per episode

learning_rate = 0.7                 # Learning rate
gamma = 0.618                       # Discounting rate

# Exploration parameters
epsilon = 1.0                       # Exploration rate
max_epsilon = 1.0                   # Exploration probability at start
min_epsilon = 0.01                  # Minimum exploration probabiltiy 
decay_rate = 0.01                   # Exponential decay rate for exploration prob

## Step 4: The Q learning algorithm
- Now we implement the Q learning algorithm:
  1. Initialize Q-values $(Q(s,a))$ arbitrarily for all state-action pairs.
  2. For life or until learning is stopped...
    1. Choose an action $(a)$ in the current world state $(s)$ based on current Q-value estimates $(Q(s,\cdot))$
    2. Take the action $(a)$ and observe the outcome state $(s^{\prime})$ and reward $(r)$
    3. Update $Q(s,a) := Q(s,a) + \alpha[r + \gamma max_{\alpha^\prime}Q(s^\prime, a^\prime) - Q(s,a)]$









In [52]:
# 2 For life or until learning is stopped
for episode in range(total_episodes):
    # Reset the environment
    state = env.reset()
    step = 0
    done = False
    
    for step in range(max_steps):
        # 3. choose an action in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0,1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])
            
        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()
            
        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action)
        
        # Bellman
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action])
        
        
        # Our new state is state
        state = new_state
        
        # If done: finish episode 
        if done == True:
            print("done at step", step)
            break
            
        episode += 1
        
        # Reduce epsilon 
        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(- decay_rate * episode)

done at step 96
done at step 79
done at step 84
done at step 56
done at step 77
done at step 60
done at step 88
done at step 43
done at step 62
done at step 60
done at step 92
done at step 81
done at step 65
done at step 81
done at step 49
done at step 46
done at step 85
done at step 94
done at step 50
done at step 27
done at step 14
done at step 68
done at step 48
done at step 21
done at step 75
done at step 39
done at step 96
done at step 49
done at step 29
done at step 85
done at step 81
done at step 41
done at step 38
done at step 74
done at step 56
done at step 96
done at step 76
done at step 93
done at step 79
done at step 30
done at step 34
done at step 32
done at step 21
done at step 30
done at step 30
done at step 12
done at step 30
done at step 44
done at step 74
done at step 17
done at step 61
done at step 56
done at step 26
done at step 62
done at step 98
done at step 41
done at step 45
done at step 48
done at step 38
done at step 64
done at step 45
done at step 36
done at 

## Step 5: Use our Q-table to play Taxi!

After 50 000 episodes, our Q-table cna be sued as a "cheatsheet" to play Taxi.

In [25]:
env.reset()
rewards = []

for episodes in range(total_test_episodes):
    state = env.reset()
    step = 0
    done = False
    total_reward = 0
    print("***************************************************************")
    print("EPISODE ", episode)
    
    for step in range(max_steps):
        env.render()
        
        action = np.argmax(qtable[state,:])
        
        new_state, reward, done, info = env.step(action)
        
        total_reward += reward
        
        if done:
            rewards.append(total_reward)
            print("Score ", total_reward)
            break
        
        state = new_state
        
env.close()
print("Score over time: " + str(sum(rewards) / total_test_episodes))

***************************************************************
EPISODE  50010
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : : : :_|
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : :_: |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : :_: : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| :_: : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
 